In [138]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [139]:
df = pd.read_csv('universities-clean.csv')


In [140]:
df.head()

,Country,University,Founded,Type,Enrollment,link
0,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/Albania
1,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/Algeria
2,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Angola
3,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/Argentina
4,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Australia


In [141]:
conn = psycopg2.connect(
    host = 'localhost',
    port = 5435,
    password = 'amdaripassword',
    user = 'amdariuser',
    database = 'amdaridb'
)

In [142]:
cursor = conn.cursor()

In [143]:
create_query_file = open('../sql/postgres_create_table.sql')
create_query = create_query_file.read()
create_query

'CREATE TABLE university (\n    id SERIAL PRIMARY KEY,\n    country VARCHAR(255) NOT NULL,\n    name VARCHAR(255) NOT NULL,\n    founded INT NOT NULL,\n    type VARCHAR(255) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    link VARCHAR(255) NOT NULL,\n\n    CONSTRAINT unique_combination UNIQUE (country, name)\n);\n\n'

In [ ]:
cursor.execute(create_query)

In [ ]:
conn.commit()

In [145]:
conn.rollback()

In [ ]:
engine = create_engine("postgresql+psycopg2://",creator=lambda: conn)

In [ ]:
pd.read_sql("SELECT * FROM university;", con=engine)

In [ ]:

data = list(df.itertuples(name=None, index=None))
merge_query_file = open('../sql/postgres_upsert.sql')
merge_query= merge_query_file.read()
merge_query



'INSERT INTO university (country,name, founded, type, enrollment, link)\nVALUES (%s, %s, %s, %s, %s, %s)\nON CONFLICT (country, name) DO UPDATE\nSET\n    founded = EXCLUDED.founded,\n    type = EXCLUDED.type,\n    enrollment = EXCLUDED.enrollment,\n    link = EXCLUDED.link;\n'

In [150]:
cursor.executemany(merge_query, data)

In [151]:
conn.commit()

In [152]:
pd.read_sql('SELECT * FROM university;', con=engine)

,id,country,name,founded,type,enrollment,link
0,70,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/Albania
1,71,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/Algeria
2,72,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Angola
3,73,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/Argentina
4,74,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Australia
...,...,...,...,...,...,...,...
64,134,Turkey,Anadolu University,1958,Public,1969733,https://en.wikipedia.org/wiki/Turkey
65,135,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/United_Kingdom
66,136,United States,Texas A&M University,1876,Public,73284,https://en.wikipedia.org/wiki/United_States
67,137,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/Uruguay
